# <center>Masakhane - Machine Translation for African Languages (Using JoeyNMT)</center>
## <leftalign> Author : Ari Ramkilowan</leftalign>
## <leftalign> Language Pair : English - isiNdebele</leftalign>
## <leftalign> Corpus : JW300 </leftalign>

<hr>

## Install JoeyNMT

In [0]:
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2199 (delta 4), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2199/2199), 2.60 MiB | 4.24 MiB/s, done.
Resolving deltas: 100% (1525/1525), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 18.1MB/s 
     |████████████████████████████████| 307kB 44.5MB/s 
     |████████████████████████████████| 215kB 37.1MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 737kB 50.2MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=4fbaf089ba87f31032bfb25477698f7b9e8d56ceae62c059790beb6d8befb44e
  Stored in directory: /tmp/pip-ephem-wheel-cache-xdbxy0qs/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml

## Mount Google Drive

In [0]:
# If running on Google Colab - mount google drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [19]:
import torch

device_num = torch.cuda.current_device()
torch.cuda.get_device_name(device_num)
# torch.cuda.is_available()

'Tesla T4'

## Set your source and target languages

In [0]:
import os
import numpy as np
import pandas as pd

source_language = "en"
target_language = "nr" 
lc = True  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted
vocab_size=4000
corpus = "JW300"

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag
os.environ["vocab_size"] = str(vocab_size)
os.environ["corpus"] = corpus

In [0]:
# This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
gdrive_path = f"/content/drive/My Drive/masakhane/{source_language}-{target_language}-{tag}/"
os.environ["gdrive_path"] = gdrive_path
! echo $gdrive_path

/content/drive/My Drive/masakhane/en-nr-baseline/


In [0]:
# create path to joeynmt executables scripts, configs etc

joey_path = f"/content/joeynmt"
os.environ["joey_path"] = joey_path
! ls $joey_path/configs

iwslt14_deen_bpe.yaml		   transformer_reverse.yaml
iwslt_deen_bahdanau.yaml	   transformer_small.yaml
iwslt_envi_luong.yaml		   transformer_wmt17_ende.yaml
iwslt_envi_xnmt.yaml		   transformer_wmt17_lven.yaml
reverse.yaml			   wmt_ende_best.yaml
small.yaml			   wmt_ende_default.yaml
transformer_copy.yaml		   wmt_lven_best.yaml
transformer_iwslt14_deen_bpe.yaml  wmt_lven_default.yaml


## Download the global test set.

***(This changes from time to time, do this just to make sure you have the most recent version)***

In [0]:
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$tgt.en
! mv test.en-$tgt.en test.en

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$tgt.$tgt 
! mv test.en-$tgt.$tgt test.$tgt

--2019-11-18 06:20:30--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2019-11-18 06:20:30 (11.6 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2019-11-18 06:20:34--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-nr.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
blanks = [] # sometimes blank lines creep innto test set - store which lines these are
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    if len(line)<=1:
      blanks.append(j)
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))
print(f'There are {len(blanks)} blank lines in the test set')

Loaded 3571 global test sentences to filter from the training/dev data.
There are 0 blank lines in the test set


In [0]:
# filter test set

source_file = f"test.{source_language}"
target_file = f"test.{target_language}"

source = []
target = []

with open(source_file) as f:
  source = f.readlines()
            
with open(target_file) as f:
  target = f.readlines()

df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

# remove trailing newline chars
df['source_sentence'] = df['source_sentence'].str.rstrip('" \n')
df['target_sentence'] = df['target_sentence'].str.rstrip('" \n')

# remove leading newline chars
df['source_sentence'] = df['source_sentence'].str.lstrip('"')
df['target_sentence'] = df['target_sentence'].str.lstrip('"')

# remove rows with really short sentences
df = df[~(df['source_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters
df = df[~(df['target_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters

# save the filtered test set
df['source_sentence'].to_csv(f'{source_file}', index=False, header=False, doublequote=False)
df['target_sentence'].to_csv(f'{target_file}', index=False, header=False, doublequote=False)

In [0]:
# copy test sets to gdrive
! cp test.$src "$gdrive_path"
! cp test.$tgt "$gdrive_path"
! cp test.$src-any.$src "$gdrive_path"

## Import prepared dataset

In [0]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
# This csv has extra columns added but no preprocessing done. all preprocessing should be captured in the NMT modelling notebook

input_file = f"{gdrive_path}/{source_language}-{target_language}-{corpus}-new.csv"
df = pd.read_csv(input_file)

In [0]:
df.head()

,source_sentence,target_sentence
0,Table of Contents,Okungaphakathi
1,"July 1 , 2010","Arhostosi 1 , 2010"
2,Do You Know God by Name ?,Umazi Kuhle na UZimu ?
3,FROM OUR COVER,EZIKHAMBISANA NESIHLOKO ESINGAPHANDLE
4,3 Can You Know God by Name ?,3 Ungamazi UZimu Ngebizo ?


In [0]:
# How many samples
size = len(df)
print(f"\n {size} samples in original text")
  


 103983 samples in original text


## Preprocess input data

In [0]:
## Preprocessing - Step 1 : Drop NaNs

df_pp = df.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping all NaNs")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102127 entries, 0 to 103982
Data columns (total 2 columns):
source_sentence    102127 non-null object
target_sentence    102127 non-null object
dtypes: object(2)
memory usage: 36.4 MB

 1856(1.78 %) samples removed by dropping all NaNs


In [0]:
## Preprocessing - Step 2a : Drop all duplicates in Source (en) text

df_pp = df_pp.drop_duplicates(subset='source_sentence')
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Source sentence duplicates")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95792 entries, 0 to 103982
Data columns (total 2 columns):
source_sentence    95792 non-null object
target_sentence    95792 non-null object
dtypes: object(2)
memory usage: 39.2 MB

 6335(6.20 %) samples removed by dropping Source sentence duplicates


In [0]:
## Preprocessing - Step 2b : Drop all duplicates in Target (zu) text

df_pp = df_pp.drop_duplicates(subset='target_sentence')
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Target sentence duplicates")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95382 entries, 0 to 103982
Data columns (total 2 columns):
source_sentence    95382 non-null object
target_sentence    95382 non-null object
dtypes: object(2)
memory usage: 41.9 MB

 410(0.43 %) samples removed by dropping Target sentence duplicates


In [0]:
##  Preprocessing - Step 3 : Remove all numeric entries

pattern = r"([0-9]*\.?[0-9]*)"  # catch integers and decimals
import re
r = re.compile(pattern)

In [0]:
%%time
##  Preprocessing - Step 3a : Remove all numeric entries - Source text

df_pp['source_sentence'] = df_pp['source_sentence'].str.replace(pattern,"")
df_pp['source_sentence'] = df_pp['source_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from source text")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95343 entries, 0 to 103982
Data columns (total 2 columns):
source_sentence    95343 non-null object
target_sentence    95343 non-null object
dtypes: object(2)
memory usage: 37.8 MB

 39(0.04 %) samples removed by dropping nummeric entries from source text
CPU times: user 1.68 s, sys: 10.2 ms, total: 1.69 s
Wall time: 1.7 s


In [0]:
%%time
##  Preprocessing - Step 3b : Remove all numeric entries - Target text

df_pp['target_sentence'] = df_pp['target_sentence'].str.replace(r,"")
df_pp['target_sentence'] = df_pp['target_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from target text")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95343 entries, 0 to 103982
Data columns (total 2 columns):
source_sentence    95343 non-null object
target_sentence    95343 non-null object
dtypes: object(2)
memory usage: 34.8 MB

 0(0.00 %) samples removed by dropping nummeric entries from target text
CPU times: user 1.73 s, sys: 20.6 ms, total: 1.75 s
Wall time: 1.77 s


#### Preprocessing - Step 4 :Get length of sentences and then drop really short sentences

In [0]:
%%time
# add length columns


df_pp['source_ch_len'] = df_pp['source_sentence'].str.len()
df_pp['source_w_len'] = [len(text.split()) for text in df_pp['source_sentence']] 
df_pp['target_ch_len'] = df_pp['target_sentence'].str.len()
df_pp['target_w_len'] = [len(text.split()) for text in df_pp['target_sentence']] 
df_pp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95343 entries, 0 to 103982
Data columns (total 6 columns):
source_sentence    95343 non-null object
target_sentence    95343 non-null object
source_ch_len      95343 non-null int64
source_w_len       95343 non-null int64
target_ch_len      95343 non-null int64
target_w_len       95343 non-null int64
dtypes: int64(4), object(2)
memory usage: 37.7 MB
CPU times: user 477 ms, sys: 0 ns, total: 477 ms
Wall time: 477 ms


In [0]:
# # character len distrn - source text - 
# df_pp['source_ch_len'].value_counts().sort_index()

In [0]:
# # character len distrn - target text
# df_pp['target_ch_len'].value_counts().sort_index()

In [0]:
## how many rows with source text <=2chars and what do they look like ?

In [0]:
# # how many single character sentences from source ?
# f"{df_pp['source_ch_len'].value_counts()[1]} single character source sentences"

# df_pp[df_pp['source_ch_len']<=1]

In [0]:
# # how many 2-character sentences from source ?
# f"{df_pp['source_ch_len'].value_counts()[2]} 2-character source sentences"

# df_pp[df_pp['source_ch_len']==2]

In [0]:
##  Preprocessing - Step 4a :  drop everything where the ch_len <=8 in source text

df_pp = df_pp[~(df_pp['source_ch_len'] <=8) ]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with source sentences <= 8 characters")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93690 entries, 0 to 103982
Data columns (total 6 columns):
source_sentence    93690 non-null object
target_sentence    93690 non-null object
source_ch_len      93690 non-null int64
source_w_len       93690 non-null int64
target_ch_len      93690 non-null int64
target_w_len       93690 non-null int64
dtypes: int64(4), object(2)
memory usage: 37.4 MB

 1653(1.73 %) samples removed by dropping rows with source sentences <= 2 characters


In [0]:
##  Preprocessing - Step 4b :  drop everything where the ch_len <=8 in target text

df_pp = df_pp[~(df_pp['target_ch_len'] <=8) ]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with target sentences <= 8 characters")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93557 entries, 0 to 103982
Data columns (total 6 columns):
source_sentence    93557 non-null object
target_sentence    93557 non-null object
source_ch_len      93557 non-null int64
source_w_len       93557 non-null int64
target_ch_len      93557 non-null int64
target_w_len       93557 non-null int64
dtypes: int64(4), object(2)
memory usage: 37.4 MB

 133(0.14 %) samples removed by dropping rows with target sentences <= 8 characters


In [0]:
%%time
##  Preprocessing - Step 5 :  remove text from test set

with open(f"{gdrive_path}/test.en-any.en") as f:
    rows = f.readlines()
test_set_en = [row.strip() for row in rows]


df_pp = df_pp[~df_pp['source_sentence'].str.strip().isin(test_set_en)]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows from test set")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92508 entries, 2 to 103982
Data columns (total 6 columns):
source_sentence    92508 non-null object
target_sentence    92508 non-null object
source_ch_len      92508 non-null int64
source_w_len       92508 non-null int64
target_ch_len      92508 non-null int64
target_w_len       92508 non-null int64
dtypes: int64(4), object(2)
memory usage: 37.1 MB

 1049(1.12 %) samples removed by dropping rows from test set
CPU times: user 212 ms, sys: 5.45 ms, total: 218 ms
Wall time: 224 ms


In [0]:
%%time
##  Preprocessing - Step 6 :  remove the extra "
df_pp['source_sentence'] = df_pp['source_sentence'].map(lambda x: x.lstrip('"').rstrip('"'))
df_pp['target_sentence'] = df_pp['target_sentence'].map(lambda x: x.lstrip('"').rstrip('"'))


df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with extra quotes")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92508 entries, 2 to 103982
Data columns (total 6 columns):
source_sentence    92508 non-null object
target_sentence    92508 non-null object
source_ch_len      92508 non-null int64
source_w_len       92508 non-null int64
target_ch_len      92508 non-null int64
target_w_len       92508 non-null int64
dtypes: int64(4), object(2)
memory usage: 37.1 MB

 0(0.00 %) samples removed by dropping rows with extra quotes
CPU times: user 237 ms, sys: 560 µs, total: 238 ms
Wall time: 241 ms


## create dev df 

In [0]:
df_dev = df_pp[['source_sentence', 'target_sentence']]
# Shuffle the data to remove bias in dev set selection.
seed=42
df_dev = df_dev.sample(frac=1, random_state=seed).reset_index(drop=True)
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92508 entries, 0 to 92507
Data columns (total 2 columns):
source_sentence    92508 non-null object
target_sentence    92508 non-null object
dtypes: object(2)
memory usage: 1.4+ MB


## Create train test dev sets

In [0]:
%%time
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_dev["source_sentence"] = df_dev["source_sentence"].str.lower()
    df_dev["target_sentence"] = df_dev["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_dev.tail(num_dev_patterns) # Herman: Error in original
stripped = df_dev.drop(df_dev.tail(num_dev_patterns).index)

with open(f"{gdrive_path}/train."+source_language, "w") as src_file, open(f"{gdrive_path}/train."+target_language, "w") as tgt_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    tgt_file.write(row["target_sentence"]+"\n")
    
with open(f"{gdrive_path}/dev."+source_language, "w") as src_file, open(f"{gdrive_path}/dev."+target_language, "w") as tgt_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    tgt_file.write(row["target_sentence"]+"\n")

CPU times: user 14.1 s, sys: 29 ms, total: 14.1 s
Wall time: 14.2 s


In [0]:
# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head "$gdrive_path/train.$src"
! echo "================================="
! head "$gdrive_path/dev.$src"
! echo "================================="
! head "$gdrive_path/test.$src"

we need similar courage in these last days 
five of them , including the manager , accepted the truth and were baptized 
your congregation also no doubt has a standing resolution to assist financially with the worldwide program of building assembly halls and kingdom halls , which benefits our brothers throughout the earth 
it’s good that you are going straight to the source 
third , they were willing to use what they had heard and understood , not only in their personal lives but also in their efforts to help others  ​ — matt 
 ,   ( a ) how did you make your decision to serve god ?
total ( in us  funds ) repaid to jehovah’s witnesses on december  ,  , by the government of france , after a  - year legal battle
paul went on to indicate that there would be others raised to heavenly life , adding : “ each one in his own proper order : christ the firstfruits , afterward those who belong to the christ during his presence  ” ​ —  cor 
as a result , , perished at jehovah’s hand 
how did jehov

In [0]:
! head "$gdrive_path/train.$tgt"
! echo "================================="
! head "$gdrive_path/dev.$tgt"
! echo "================================="
! head "$gdrive_path/test.$tgt"

sitlhoga isibindi esifanako emihleni yokuphela le 
abahlanu babo , kuhlanganise nemenenjara , bamukele iqiniso babhabhadiswa 
nawe kungenzeka ibandla lakho liqunte ukukhupha imali ethileko  imali leyo isekela ihlelo lomsebenzi wokwakha amawolo wemihlangano emincani nemikhulu  amawolo la azuzisa abazalwana ephasini loke 
kuhle ngombana ubuze emthonjeni ofaneleko 
kwesithathu , lokhu abafundileko bebangafuni ukukusebenzisa emaphilwenabo kwaphela kodwana bebasiza nabanye ngakho  — mat 
 ,   ( a ) wasenza njani isiqunto sakho sokukhonza uzimu ?
inani elipheleleko elabuyiselwa kibofakazi bakajehova ngodesemba  ,  , libuyiselwa ngurhulumende wefrance , ngemva kwepi engokomthetho yeminyaka eli - 
upowula waragela phambili wahlathulula ukuthi kunabanye abazokuvuselwa ezulwini : “ omunye nomunye ngedlhego lakhe : kuzokuthoma ukrestu ntangi bese kuthi nabuyako , kuvuke labo abangebakhe  ” —  kor 
umphumela waba kukuthi ujehova amadlhe abazi -   zabantwabo 
ujehova wamsiza njani udavida bona akgh

## Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
%%time
! subword-nmt learn-joint-bpe-and-vocab --input  "$gdrive_path"train.$src  "$gdrive_path"train.$tgt -s $vocab_size -o  "$gdrive_path"bpe.codes.$vocab_size --write-vocabulary  "$gdrive_path"vocab.$src  "$gdrive_path"vocab.$tgt

# Apply BPE splits to the train, development and test data.
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"train.$src > "$gdrive_path"train.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"train.$tgt > "$gdrive_path"train.bpe.$tgt

! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"dev.$src > "$gdrive_path"dev.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"dev.$tgt > "$gdrive_path"dev.bpe.$tgt

! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"test.$src > "$gdrive_path"test.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"test.$tgt > "$gdrive_path"test.bpe.$tgt


CPU times: user 380 ms, sys: 82.1 ms, total: 462 ms
Wall time: 1min 12s


In [0]:
# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py "$gdrive_path"train.bpe."$src" "$gdrive_path"train.bpe."$tgt" --output_path "$gdrive_path"vocab.txt

In [0]:
# Some output
! echo "BPE Ndebele Sentences"
! tail -n 5 "$gdrive_path"test.bpe.$tgt

! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path"vocab.txt  # Herman


BPE Ndebele Sentences
L@@ okho kwenza bona ngaz@@ iwe njengom@@ untu ong@@ akath@@ embek@@ i .
"@@ K@@ wathi bona ng@@ ifunde iqiniso , akhenge ngis@@ av@@ uma uku@@ r@@ aga n@@ ent@@ wel@@ ey@@ o , nanyana umb@@ ereg@@ o lo bew@@ ub@@ had@@ ela kangaka .@@ "
"@@ N@@ g@@ isibonelo esihle eb@@ as@@ any@@ an@@ eni bami ababili , begodu ng@@ ikgh@@ ona noku@@ b@@ ereg@@ iswa ebandleni .@@ "
E@@ b@@ ant@@ wini abab@@ had@@ el@@ isa umth@@ elo nak@@ il@@ abo eng@@ ib@@ ereg@@ isana nabo ngaz@@ iwa njengom@@ untu oth@@ embekileko . ”
U@@ R@@ ute waf@@ ud@@ ukela kw@@ a - I@@ s@@ rayeli lapho ebek@@ azoku@@ kgh@@ ona uku@@ lotjha khona u@@ Z@@ imu weqiniso .
Combined BPE Vocab
wha@@
prophec@@
espe@@
ething
nex@@
haps
probl@@
ʺ
uld
+


## Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
name = '%s%s%s%s' % (source_language, target_language, str(vocab_size),tag)
name

'ennr4000baseline'

In [0]:
# Create this dir before we run for the first time so we store check points
# !mkdir -p "$gdrive_path/pretrained/$src$tgt$vocab_size$tag/" # Herman

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s%s%s' % (source_language, target_language, str(vocab_size),tag)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}train.bpe"
    dev:   "{gdrive_path}dev.bpe"
    test:  "{gdrive_path}test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}vocab.txt"
    trg_vocab: "{gdrive_path}vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{gdrive_path}pretrained/{name}/46000.ckpt"
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer/{name}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.3
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.4
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.3
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.4
""".format(name=name,
           gdrive_path=os.environ["gdrive_path"],
           source_language=source_language,
           target_language=target_language
          )

with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

## Train the Model

In [0]:
! cd joeynmt/configs; ls
# copy config to gdrive
! cp joeynmt/configs/transformer_$src$tgt$vocab_size$tag.yaml "$gdrive_path/pretrained/$src$tgt$vocab_size$tag/"

iwslt14_deen_bpe.yaml		   transformer_reverse.yaml
iwslt_deen_bahdanau.yaml	   transformer_small.yaml
iwslt_envi_luong.yaml		   transformer_wmt17_ende.yaml
iwslt_envi_xnmt.yaml		   transformer_wmt17_lven.yaml
reverse.yaml			   wmt_ende_best.yaml
small.yaml			   wmt_ende_default.yaml
transformer_copy.yaml		   wmt_lven_best.yaml
transformer_ennr4000baseline.yaml  wmt_lven_default.yaml
transformer_iwslt14_deen_bpe.yaml


In [0]:
%%time
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
! cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt$vocab_size$tag.yaml

2019-11-28 13:10:22,179 Hello! This is Joey-NMT.
2019-11-28 13:10:23,931 Total params: 12118528
2019-11-28 13:10:23,933 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp  -r joeynmt/models/${src}${tgt}${vocab_size}${tag}_transformer/* "$gdrive_path""pretrained/$src$tgt$vocab_size$tag/"
!cp  joeynmt/models/${src}${tgt}${vocab_size}${tag}_transformer/best.ckpt "$gdrive_path""pretrained/$src$tgt$vocab_size$tag"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-nr-baseline/pretrained/ennr4000baseline/best.ckpt': Function not implemented


In [0]:
# Copy the created models from t google drive to joeynmt folder
!cp  -r "$gdrive_path""pretrained/$src$tgt$vocab_size$tag/" "joeynmt/models/$src$tgt$vocab_size$tag""_transformer/"
# !cp  joeynmt/models/${src}${tgt}${vocab_size}${tag}_transformer/best.ckpt "$gdrive_path""pretrained/$src$tgt$vocab_size$tag"

In [0]:
# copy across the config file
!cp  joeynmt/configs/transformer_${src}${tgt}${vocab_size}${tag}.yaml "$gdrive_path"

In [0]:
# Test our model
# ! cd joeynmt; python3 -m joeynmt test "$gdrive_path""pretrained/$src$tgt$vocab_size$tag/config.yaml"

In [0]:
# OR
! cd joeynmt; python3 -m joeynmt test "$gdrive_path""transformer_${src}${tgt}${vocab_size}${tag}.yaml"

2019-11-28 13:37:38,730 Hello! This is Joey-NMT.
2019-11-28 13:38:08,636  dev bleu:  14.93 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-11-28 13:39:12,496 test bleu:   4.01 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [0]:
!touch "$gdrive_path"sample_input.txt
!echo 'Hello there' > "$gdrive_path"sample_input.txt

In [0]:
# BPE some bespoke input
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"sample_input.txt > "$gdrive_path"sample_input_bped.txt

In [0]:
! more "$gdrive_path"sample_input_bped.txt

H@@ el@@ lo there


In [0]:
# Translate mode is mopre interactive but almsot the same as running in test mode
! cd joeynmt; python3 -m joeynmt translate "$gdrive_path""transformer_${src}${tgt}${vocab_size}${tag}.yaml"

2019-11-28 13:57:52,461 Hello! This is Joey-NMT.

Please enter a source sentence (pre-processed): 
H@@ el@@ lo there
JoeyNMT: u - ella ukhona

Please enter a source sentence (pre-processed): 
H@@ el@@ lo
JoeyNMT: u - ella

Please enter a source sentence (pre-processed): 
h@@ el@@ lo
JoeyNMT: hella

Please enter a source sentence (pre-processed): 

Bye.


In [0]:
# Output our validation accuracy
! cat "$gdrive_path/pretrained/${src}${tgt}${vocab_size}${tag}/validations.txt"

Steps: 19000	Loss: 52407.53906	PPL: 9.50908	bleu: 10.34544	LR: 0.00030000	*
Steps: 20000	Loss: 51819.73047	PPL: 9.27188	bleu: 10.89485	LR: 0.00030000	*
Steps: 21000	Loss: 51056.16797	PPL: 8.97256	bleu: 10.77206	LR: 0.00030000	*
Steps: 22000	Loss: 50872.08984	PPL: 8.90186	bleu: 11.24929	LR: 0.00030000	*
Steps: 23000	Loss: 50294.21484	PPL: 8.68351	bleu: 11.40370	LR: 0.00030000	*
Steps: 24000	Loss: 49921.73047	PPL: 8.54561	bleu: 11.70847	LR: 0.00030000	*
Steps: 25000	Loss: 49970.34766	PPL: 8.56349	bleu: 11.78933	LR: 0.00030000	
Steps: 26000	Loss: 49318.21484	PPL: 8.32682	bleu: 12.43307	LR: 0.00030000	*
Steps: 27000	Loss: 48703.39844	PPL: 8.10969	bleu: 12.78421	LR: 0.00030000	*
Steps: 28000	Loss: 48488.24219	PPL: 8.03505	bleu: 12.43170	LR: 0.00030000	*
Steps: 29000	Loss: 48164.97656	PPL: 7.92419	bleu: 12.85117	LR: 0.00030000	*
Steps: 30000	Loss: 47853.74219	PPL: 7.81891	bleu: 13.07397	LR: 0.00030000	*
Steps: 31000	Loss: 47605.07031	PPL: 7.73579	bleu: 13.14769	LR: 0.00030000	*
Steps: 32000	